In [59]:
import pandas as pd

! python extract_ossc_formulas.py \
  --sheet-id 1U4XKFcUypBs0ruJybO9oXKNgIs6_fUDATj4U7gjuZr4 \
  --gid 554372652 \
  --out-dir out/ \
  --source "Open Source Smell Culture – Formulas Database" \
  --source-url "https://docs.google.com/spreadsheets/d/1U4XKFcUypBs0ruJybO9oXKNgIs6_fUDATj4U7gjuZr4/edit?gid=554372652#gid=554372652" \
  --license "CC BY-SA 4.0"


In [9]:
! python Desktop/LunarAIccord/helpers/extract_ossc_formulas.py \
  --sheet-id 1U4XKFcUypBs0ruJybO9oXKNgIs6_fUDATj4U7gjuZr4 \
  --gid 554372652 \
  --out-dir /Users/ranykhirbawi/Desktop/my_csv.csv \
  --source "Open Source Smell Culture – Formulas Database" \
  --source-url "https://docs.google.com/spreadsheets/d/1U4XKFcUypBs0ruJybO9oXKNgIs6_fUDATj4U7gjuZr4/edit?gid=554372652#gid=554372652" \
  --license "CC BY-SA 4.0"


In [11]:
!python Desktop/LunarAIccord/helpers/extract_ossc_formulas.py \
  --sheet-id 1U4XKFcUypBs0ruJybO9oXKNgIs6_fUDATj4U7gjuZr4 \
  --gid 554372652 \
  --out-dir /Users/ranykhirbawi/Desktop/LunarAIccord

Wrote 1 formulas to:
 - /Users/ranykhirbawi/Desktop/LunarAIccord/formulas.json
 - /Users/ranykhirbawi/Desktop/LunarAIccord/formulas.jsonl


In [12]:
import requests
u="https://docs.google.com/spreadsheets/d/1U4XKFcUypBs0ruJybO9oXKNgIs6_fUDATj4U7gjuZr4/export?format=csv&gid=554372652"
t=requests.get(u,headers={'User-Agent':'Mozilla/5.0'}).text
print(len(t), t[:200])


8977 <!DOCTYPE html><style nonce="-y2lYacG5ONCM85kEW5law">body{height:100%;margin:0;width:100%}@media (max-height:350px){.button{font-size:10px}.button-container{margin-top:16px}.button.primary-button,.but


In [ ]:
# # %% [markdown]
# # Lunar AIccord — offline MVP scaffold (writes files to ./lunar-aiccord)
# # Requirements: Python 3.11+, pandas, numpy (no external APIs needed)

# # %%
# import os, textwrap, shutil, json
# from pathlib import Path

# BASE = Path("/Users/ranykhirbawi/Desktop/LunarAIccord")
# AGENT = BASE/"agent"
# INGEST = BASE/"data_ingest"
# EXAMPLES = BASE/"examples"
# OUTPUTS = EXAMPLES/"outputs"
# INDICES = BASE/"indices"
# TOKENS = BASE/"tokens_count"

# # Clean slate (CAREFUL if you modified files locally)
# if BASE.exists():
#     shutil.rmtree(BASE)
# for p in (AGENT, INGEST, EXAMPLES, OUTPUTS, INDICES, TOKENS):
#     p.mkdir(parents=True, exist_ok=True)

# def w(path: Path, content: str):
#     path.parent.mkdir(parents=True, exist_ok=True)
#     path.write_text(textwrap.dedent(content).strip()+"\n", encoding="utf-8")

In [90]:
print(AGENT)

/Users/ranykhirbawi/Desktop/LunarAIccord/agent


In [87]:


# ---------------- core metadata files ----------------
w(BASE/"requirements.txt", """
pandas
numpy
python-dotenv
langchain
""")

w(BASE/".env.example", "API_KEY=\n")

w(EXAMPLES/"01_brief.txt", """
Create a tranquil summer lobby signature for a spa brand. 
Use calm, airy vibes; hints of sea breeze. The brand colors are teal and sand.
""")
w(EXAMPLES/"02_brief.txt", """
A winter scent for a luxury menswear boutique: confident, polished, evening wear.
Prefer warm woods and a touch of leather. Brand colors: black and gold.
""")
w(EXAMPLES/"03_brief.txt", """
Fresh, clean fragrance for a wellness app launch. Minimalist, green, uplifting.
Avoid heavy animalic notes. Colors: soft green and white.
""")
w(TOKENS/"total_tokens.txt", "0")

# ---------------- agent/local_utils.py ----------------
w(AGENT/"local_utils.py", r"""
import re, math
from collections import Counter

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    s = s.lower()
    s = re.sub(r"[^a-z0-9#]+", " ", s)
    return " ".join(s.split())

def tokenize(s: str):
    s = normalize_text(s)
    return [t for t in s.split() if t]

class TinyTfidf:
    def __init__(self):
        self.vocab = {}
        self.idf = []
        self.fitted = False

    def fit(self, corpus):
        docs = [tokenize(x) for x in corpus]
        vocab = {}
        for doc in docs:
            for t in set(doc):
                if t not in vocab:
                    vocab[t] = len(vocab)
        N = len(docs)
        df = [0]*len(vocab)
        for doc in docs:
            seen = set(doc)
            for t in seen:
                df[vocab[t]] += 1
        idf = [math.log((N+1)/(dfi+1)) + 1.0 for dfi in df]
        self.vocab = vocab
        self.idf = idf
        self.fitted = True
        return self

    def transform(self, corpus):
        if not self.fitted:
            raise RuntimeError("TinyTfidf not fitted")
        X = []
        for text in corpus:
            tokens = tokenize(text)
            counts = Counter(tokens)
            vec = [0.0]*len(self.vocab)
            for t, c in counts.items():
                j = self.vocab.get(t)
                if j is not None:
                    vec[j] = (c) * self.idf[j]
            norm = math.sqrt(sum(v*v for v in vec)) or 1.0
            vec = [v/norm for v in vec]
            X.append(vec)
        return X

def cosine(u, v):
    return sum(a*b for a,b in zip(u,v))

def flatten_json(obj, prefix=""):
    parts = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            parts.append(f"{k}: {flatten_json(v, k)}")
    elif isinstance(obj, list):
        parts.append(", ".join([flatten_json(x, prefix) for x in obj]))
    else:
        parts.append(str(obj))
    return " ".join(str(p) for p in parts if p is not None)

def map_hex_to_color_words(hex_code: str):
    try:
        if not hex_code.startswith("#") or len(hex_code) != 7:
            return []
        r = int(hex_code[1:3], 16)/255.0
        g = int(hex_code[3:5], 16)/255.0
        b = int(hex_code[5:7], 16)/255.0
        mx, mn = max(r,g,b), min(r,g,b)
        d = mx - mn
        if d == 0:
            h = 0
        elif mx == r:
            h = (60 * ((g - b)/d) + 360) % 360
        elif mx == g:
            h = (60 * ((b - r)/d) + 120) % 360
        else:
            h = (60 * ((r - g)/d) + 240) % 360
        if 170 <= h <= 250:
            return ["blue", "marine"]
        if 80 <= h <= 160:
            return ["green"]
        if 20 <= h <= 50:
            return ["gold", "amber"]
        if h < 20 or h > 300:
            return ["red", "warm"]
        if 250 < h <= 300:
            return ["violet"]
        return []
    except Exception:
        return []

FAMILY_TO_ROLE = {
    "citrus": "top", "green": "top", "aldehydic": "top", "marine": "top",
    "lactonic": "mid", "floral": "mid", "spicy": "mid", "herbal": "mid",
    "woody": "base", "amber": "base", "musk": "base", "leathery": "base",
    "oud": "base", "balsamic": "base", "incense": "base",
}

def pick_role_from_family(family: str):
    if not family:
        return "mid"
    fam = normalize_text(family).replace("/", " ").split()[0]
    return FAMILY_TO_ROLE.get(fam, "mid")
""")

# ---------------- data_ingest/build_collections.py ----------------
w(INGEST/"build_collections.py", r"""
import os, json, pickle, pandas as pd
from pathlib import Path
from agent.local_utils import TinyTfidf, normalize_text, flatten_json

BASE = Path(__file__).resolve().parents[1]
INDICES = BASE / "indices"

# Data directory (put your four files here)
DATA_DIR = Path(os.environ.get("LUNAR_DATA_DIR", "data"))
MATERIALS = DATA_DIR/"materials_catalog.jsonl"
FORMULAS  = DATA_DIR/"formulas.jsonl"
NOTES1    = DATA_DIR/"nlp_notes_data.csv"
NOTES2    = DATA_DIR/"fra_cleaned.csv"

def read_jsonl(path: Path):
    items = []
    if not path.exists():
        return items
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                items.append(json.loads(line))
            except Exception:
                try:
                    items.append(json.loads(line.rstrip(", ")))
                except Exception:
                    pass
    return items

def extract_material_record(raw):
    name = raw.get("material_name") or raw.get("name") or raw.get("material") or raw.get("Name") or raw.get("Material")
    if not name:
        for _,v in raw.items():
            if isinstance(v, str) and len(v) > 1:
                name = v; break
    family = raw.get("family") or raw.get("Family") or raw.get("olfactive_family") or raw.get("Olfactive family")
    descriptors = raw.get("descriptors") or raw.get("description") or raw.get("notes") or raw.get("Descriptors")
    tenacity = raw.get("tenacity") or raw.get("Tenacity")
    usage_max = raw.get("usage_max") or raw.get("ifra_max") or raw.get("max_usage")
    allergens = raw.get("allergens") or raw.get("Allergens")
    potency = raw.get("strength") or raw.get("potency") or raw.get("intensity")
    text_blob = " ; ".join([
        str(name or ""),
        f"family={family}" if family else "",
        f"tenacity={tenacity}" if tenacity else "",
        f"potency={potency}" if potency else "",
        f"descriptors={descriptors}" if descriptors else "",
        f"usage_max={usage_max}" if usage_max else "",
        f"allergens={allergens}" if allergens else "",
    ])
    return {
        "name": name or "unknown-material",
        "family": family,
        "descriptors": descriptors,
        "tenacity": tenacity,
        "potency": potency,
        "usage_max": usage_max,
        "allergens": allergens,
        "text": text_blob
    }

def load_notes_csv(path: Path):
    rows = []
    if not path.exists():
        return rows
    df = pd.read_csv(path)
    cols = list(df.columns)
    name_candidates = [c for c in cols if "note" in c.lower() or "name" in c.lower() or c.lower() in ("material","ingredient")]
    family_candidates = [c for c in cols if "family" in c.lower()]
    desc_candidates = [c for c in cols if "desc" in c.lower() or "descriptor" in c.lower() or "info" in c.lower() or "text" in c.lower() or "synopsis" in c.lower()]
    name_col = name_candidates[0] if name_candidates else cols[0]
    fam_col  = family_candidates[0] if family_candidates else None
    des_col  = desc_candidates[0] if desc_candidates else None
    for _, r in df.iterrows():
        name = str(r.get(name_col, "")).strip()
        if not name: 
            continue
        family = str(r.get(fam_col, "")).strip() if fam_col else ""
        desc = str(r.get(des_col, "")).strip() if des_col else ""
        text = " ; ".join([name, f"family={family}" if family else "", desc])
        rows.append({"note": name, "family": family, "description": desc, "text": text})
    return rows

def main():
    INDICES.mkdir(exist_ok=True, parents=True)

    # Materials
    materials_raw = read_jsonl(MATERIALS)
    materials = [extract_material_record(x) for x in materials_raw]
    materials_corpus = [x["text"] for x in materials] if materials else ["placeholder"]
    mat_vec = TinyTfidf().fit(materials_corpus)
    mat_X = mat_vec.transform(materials_corpus)
    pickle.dump({"vectorizer": mat_vec, "X": mat_X, "rows": materials}, open(INDICES/"materials.pkl","wb"))

    # Notes
    notes = load_notes_csv(NOTES1) + load_notes_csv(NOTES2)
    notes_corpus = [x["text"] for x in notes] if notes else ["placeholder"]
    note_vec = TinyTfidf().fit(notes_corpus)
    note_X = note_vec.transform(notes_corpus)
    pickle.dump({"vectorizer": note_vec, "X": note_X, "rows": notes}, open(INDICES/"notes.pkl","wb"))

    # Formulas
    formulas_raw = read_jsonl(FORMULAS)
    formulas_rows = []
    for fr in formulas_raw:
        text = flatten_json(fr)
        formulas_rows.append({"raw": fr, "text": text})
    formulas_corpus = [x["text"] for x in formulas_rows] if formulas_rows else [
        "top bergamot 2 aldehydes 0.3 mid rose 4 jasmine 2 base cedar 3 amber 2 musk 0.5"
    ]
    form_vec = TinyTfidf().fit(formulas_corpus)
    form_X = form_vec.transform(formulas_corpus)
    pickle.dump({"vectorizer": form_vec, "X": form_X, "rows": formulas_rows}, open(INDICES/"formulas.pkl","wb"))

    # Global vectorizer (for evaluator)
    global_corpus = materials_corpus + notes_corpus + formulas_corpus + [
        "calm airy sea breeze marine tranquil luminous confident polished green fresh clean warm woody leathery musk amber balsamic incense citrus floral spicy herbal aldehydic lactonic"
    ]
    glob_vec = TinyTfidf().fit(global_corpus)
    pickle.dump({"vectorizer": glob_vec}, open(INDICES/"global.pkl","wb"))

    print("Indices built in", INDICES)

if __name__ == "__main__":
    main()
""")

# ---------------- agent/intent_parser.py ----------------
w(AGENT/"intent_parser.py", r"""
import re
from dataclasses import dataclass
from typing import List, Optional
from agent.local_utils import normalize_text

SEASONS = ["spring","summer","autumn","fall","winter","all-season"]
CONTEXTS = ["spa","gym","office","hotel","retail","boutique","menswear","womenswear","wellness","lobby","app","lounge","event"]

@dataclass
class Intent:
    season: Optional[str]
    context: Optional[str]
    target_audience: Optional[str]
    brand_tone: List[str]
    constraints: List[str]
    color_terms: List[str]
    raw: str

def parse_colors(raw: str, extra_colors: List[str]) -> List[str]:
    color_words = []
    color_words += re.findall(r"#(?:[0-9a-fA-F]{6})", raw)
    named = re.findall(r"\b(black|white|gold|amber|teal|green|blue|red|violet|purple|pink|sand|beige)\b", raw)
    color_words += [c.lower() for c in named]
    color_words += [c.lower() for c in extra_colors if c]
    seen = set(); out = []
    for c in color_words:
        if c not in seen:
            out.append(c); seen.add(c)
    return out

def parse_intent(brief: str, colors: List[str]) -> Intent:
    txt = normalize_text(brief)
    season = next((s for s in SEASONS if s in txt), None)
    context = next((c for c in CONTEXTS if c in txt), None)
    target = None
    if "menswear" in txt or " men " in f" {txt} ":
        target = "men"
    elif "womenswear" in txt or " women " in f" {txt} ":
        target = "women"
    tone = [w for w in txt.split() if w.endswith("y") or w.endswith("al") or w in ("calm","fresh","clean","confident","polished","minimalist","uplifting","warm","airy","marine")]
    constraints = []
    if "avoid animalic" in txt or "no animalic" in txt:
        constraints.append("no_animalic")
    color_terms = parse_colors(txt, colors)
    return Intent(season, context, target, tone, constraints, color_terms, raw=brief)
""")

# ---------------- agent/sensory_moodboard.py ----------------
w(AGENT/"sensory_moodboard.py", r"""
from dataclasses import dataclass
from typing import List
from agent.local_utils import map_hex_to_color_words, normalize_text

@dataclass
class SensorySeed:
    seed_notes: List[str]
    emotion_text: str
    emotion_terms: List[str]

COLOR_HINTS = {
    "teal": ["marine","green","fresh"],
    "gold": ["amber","warm","luminous"],
    "black": ["leathery","smoky","dark"],
    "white": ["clean","airy","minimalist"],
    "green": ["green","herbal","fresh"],
    "blue": ["marine","cool","calm"],
    "sand": ["amber","beige","soft"],
    "amber": ["amber","warm"],
}

def build_seed(intent_tone: List[str], color_terms: List[str]) -> SensorySeed:
    hints = []
    for c in color_terms:
        if c.startswith("#"):
            hints += map_hex_to_color_words(c)
        else:
            hints += COLOR_HINTS.get(c, [])
    uniq = []
    for w in (hints + intent_tone):
        w = normalize_text(w)
        if w and w not in uniq:
            uniq.append(w)
    base_emotions = [w for w in uniq if w in ("calm","fresh","clean","warm","airy","marine","luminous","minimalist","uplifting","confident","polished","soft","cool","dark")]
    emotion_text = " ".join(base_emotions[:6]) or "balanced modern clean"
    return SensorySeed(seed_notes=uniq[:12], emotion_text=emotion_text, emotion_terms=uniq)
""")

# ---------------- agent/note_rag.py ----------------
w(AGENT/"note_rag.py", r"""
import pickle
from dataclasses import dataclass
from typing import List, Dict
from pathlib import Path
from agent.local_utils import cosine, normalize_text

BASE = Path(__file__).resolve().parents[1]
INDICES = BASE / "indices"

@dataclass
class Candidate:
    name: str
    family: str
    meta: Dict
    score: float

def _top_k(vec, X, rows, k=20):
    sims = [(i, cosine(vec, x)) for i, x in enumerate(X)]
    sims.sort(key=lambda x: x[1], reverse=True)
    out = []
    for i, s in sims[:k]:
        r = rows[i]
        name = r.get("name") or r.get("note") or r.get("text","")[:40]
        family = r.get("family") or ""
        out.append(Candidate(name=name, family=family, meta=r, score=float(s)))
    return out

def retrieve_candidates(seed_terms: List[str], emotion_text: str, top_k=24):
    with open(INDICES/"notes.pkl", "rb") as f:
        notes = pickle.load(f)
    with open(INDICES/"materials.pkl", "rb") as f:
        mats = pickle.load(f)
    query = " ".join(seed_terms + [emotion_text])
    q_notes = notes["vectorizer"].transform([query])[0]
    q_mats  = mats["vectorizer"].transform([query])[0]
    cand_notes = _top_k(q_notes, notes["X"], notes["rows"], k=top_k//2)
    cand_mats  = _top_k(q_mats,  mats["X"],  mats["rows"],  k=top_k//2)
    merged = {}
    for c in cand_notes + cand_mats:
        key = normalize_text(c.name)
        if key not in merged or c.score > merged[key].score:
            merged[key] = c
    return sorted(merged.values(), key=lambda x: x.score, reverse=True)
""")

# ---------------- agent/scent_composer.py ----------------
w(AGENT/"scent_composer.py", r"""
from dataclasses import dataclass
from typing import List, Dict
from agent.local_utils import pick_role_from_family

@dataclass
class MaterialUse:
    name: str
    role: str
    percent: float
    info: Dict

@dataclass
class Formula:
    top: List[MaterialUse]
    mid: List[MaterialUse]
    base: List[MaterialUse]

def allocate_roles(candidates, max_per_role=5):
    buckets = {"top":[], "mid":[], "base":[]}
    for c in candidates:
        fam = c.family or (c.meta.get("family") if isinstance(c.meta, dict) else "")
        role = pick_role_from_family(fam)
        buckets[role].append(c)
    # backfill if empty
    allc = candidates[:]
    for r in ("top","mid","base"):
        if not buckets[r] and allc:
            buckets[r].append(allc[0])
    for r in buckets:
        buckets[r] = buckets[r][:max_per_role]
    return buckets

def percent_plan(role_counts):
    split = {"top":0.30, "mid":0.40, "base":0.30}
    plan = {}
    for r, cnt in role_counts.items():
        if cnt <= 0:
            plan[r] = []
            continue
        share = split[r]*100.0
        weights = [1.0/(i+1) for i in range(cnt)]
        s = sum(weights)
        plan[r] = [round(share*(w/s), 2) for w in weights]
    return plan

def compose(candidates) -> Formula:
    buckets = allocate_roles(candidates)
    plan = percent_plan({r:len(buckets[r]) for r in buckets})
    def mk(role):
        out = []
        for i, c in enumerate(buckets[role]):
            out.append(MaterialUse(
                name=c.name, role=role, percent=plan[role][i] if i < len(plan[role]) else 0.0, info=c.meta
            ))
        return out
    return Formula(top=mk("top"), mid=mk("mid"), base=mk("base"))
""")

# ---------------- agent/compliance_agent.py ----------------
w(AGENT/"compliance_agent.py", r"""
from dataclasses import dataclass
from typing import List, Dict

@dataclass
class ComplianceResult:
    ok: bool
    warnings: List[str]
    fixes: List[Dict]

def check(formula):
    warnings = []
    fixes = []
    for role_list in [formula.top, formula.mid, formula.base]:
        for mu in role_list:
            meta = mu.info if isinstance(mu.info, dict) else {}
            usage_max = meta.get("usage_max")
            try:
                if usage_max is not None:
                    if isinstance(usage_max, str) and "%" in usage_max:
                        um = float(usage_max.replace("%","").strip())
                    else:
                        um = float(usage_max)
                    if mu.percent > um:
                        fixes.append({"material": mu.name, "from": mu.percent, "to": um, "reason": "usage_max clamp"})
                if meta.get("allergens"):
                    warnings.append(f"{mu.name}: allergens listed: {meta.get('allergens')}")
            except Exception:
                continue
    ok = len([fx for fx in fixes if fx["to"] < fx["from"]]) == 0
    return ComplianceResult(ok=ok, warnings=warnings, fixes=fixes)

def apply_fixes(formula, fixes):
    target = {fx["material"]: fx["to"] for fx in fixes if fx.get("to") is not None}
    if not target:
        return formula
    all_items = formula.top + formula.mid + formula.base
    for mu in all_items:
        if mu.name in target:
            mu.percent = target[mu.name]
    total = sum(mu.percent for mu in all_items) or 1.0
    for mu in all_items:
        mu.percent = round(100.0 * mu.percent / total, 2)
    return formula
""")

# ---------------- agent/evaluator.py ----------------
w(AGENT/"evaluator.py", r"""
import pickle
from dataclasses import dataclass
from pathlib import Path

BASE = Path(__file__).resolve().parents[1]
INDICES = BASE / "indices"

@dataclass
class EvalResult:
    score: float
    rationale: str

def formula_to_text(formula):
    def part(lst):
        return ", ".join([f"{x.name} {x.percent}%" for x in lst])
    return f"top: {part(formula.top)} | mid: {part(formula.mid)} | base: {part(formula.base)}"

def evaluate(formula, emotion_text: str) -> EvalResult:
    with open(INDICES/"global.pkl", "rb") as f:
        glob = pickle.load(f)
    vec = glob["vectorizer"]
    ftxt = formula_to_text(formula)
    q = vec.transform([emotion_text])[0]
    d = vec.transform([ftxt])[0]
    score = sum(a*b for a,b in zip(q,d))
    s = max(0.0, min(1.0, score))
    scaled = round(40.0 + 55.0 * s, 1)
    rationale = "Alignment via TF-IDF cosine on local corpus."
    return EvalResult(score=scaled, rationale=rationale)
""")

# ---------------- agent/brand_styler.py ----------------
w(AGENT/"brand_styler.py", r"""
import random
from dataclasses import dataclass

@dataclass
class Branding:
    name: str
    story: str

ADJ = ["Lunar","Equinox","Nocturne","Aurea","Aqua","Verde","Obsidian","Nimbus","Serene","Solstice"]
NUUN = ["Accord","Veil","Current","Silhouette","Tide","Ember","Whisper","Halo","Pulse","Drift"]

def style(intent, formula) -> Branding:
    base = random.choice(ADJ) + " " + random.choice(NUUN)
    tone = ", ".join(list(dict.fromkeys(intent.brand_tone[:4]))) or "modern, clean"
    line = f"{base} captures a {tone} aura, translating your brief into an elegant balance of top, heart, and base."
    return Branding(name=base, story=line)
""")

# ---------------- agent/orchestrator.py ----------------
w(AGENT/"orchestrator.py", r"""
from pathlib import Path
from agent.intent_parser import parse_intent
from agent.sensory_moodboard import build_seed
from agent.note_rag import retrieve_candidates
from agent.scent_composer import compose
from agent.compliance_agent import check, apply_fixes
from agent.evaluator import evaluate
from agent.brand_styler import style

BASE = Path(__file__).resolve().parents[1]
OUTPUTS = (BASE / "examples" / "outputs")
OUTPUTS.mkdir(exist_ok=True, parents=True)

def generate(brief: str, colors=None):
    colors = colors or []
    intent = parse_intent(brief, colors)
    seed = build_seed(intent.brand_tone, intent.color_terms)
    cands = retrieve_candidates(seed.seed_notes, seed.emotion_text, top_k=24)
    formula = compose(cands)
    comp = check(formula)
    if comp.fixes:
        formula = apply_fixes(formula, comp.fixes)
    evalr = evaluate(formula, seed.emotion_text)
    brand = style(intent, formula)
    res = {
        "intent": vars(intent),
        "seed": {"seed_notes": seed.seed_notes, "emotion_text": seed.emotion_text},
        "formula": {
            "top": [vars(x) for x in formula.top],
            "mid": [vars(x) for x in formula.mid],
            "base": [vars(x) for x in formula.base],
        },
        "compliance": {
            "ok": comp.ok,
            "warnings": comp.warnings,
            "fixes": comp.fixes,
        },
        "evaluation": {"score": evalr.score, "rationale": evalr.rationale},
        "branding": {"name": brand.name, "story": brand.story},
    }
    return res

def save_output(res: dict, name: str):
    import json
    out_json = OUTPUTS / f"{name}.json"
    out_md   = OUTPUTS / f"{name}.md"
    out_json.write_text(json.dumps(res, indent=2, ensure_ascii=False))
    def part(lst):
        return ", ".join([f"{x['name']} {x['percent']}%" for x in lst])
    md = (
    "# Lunar AIccord — {name}\n\n"
    "**Score:** {score} / 100  \n"
    "**Story:** {story}\n\n"
    "## Formula\n"
    "- **Top:** {top}\n"
    "- **Mid:** {mid}\n"
    "- **Base:** {base}\n\n"
    "## Compliance\n"
    "- OK: {ok}\n"
    "- Warnings: {warnings}\n"
    "- Fixes: {fixes}\n"
    ).format(
        name=res['branding']['name'],
        score=res['evaluation']['score'],
        story=res['branding']['story'],
        top=part(res['formula']['top']),
        mid=part(res['formula']['mid']),
        base=part(res['formula']['base']),
        ok=res['compliance']['ok'],
        warnings="; ".join(res['compliance']['warnings']) or "None",
        fixes=res['compliance']['fixes'] or "None",
    )

    out_md.write_text(md)
    return str(out_json), str(out_md)
""")


# ---------------- run.py ----------------
w(BASE/"run.py", r"""
import argparse, os
from pathlib import Path
from agent.orchestrator import generate, save_output
from data_ingest.build_collections import main as build_indices

def parse_colors_arg(s: str):
    if not s:
        return []
    parts = [x.strip() for x in s.split(",")]
    return [p for p in parts if p]

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--brief", required=True, help="Path to a text file containing the brief")
    ap.add_argument("--colors", default="", help="Comma-separated color terms or hex (e.g., \"#2a5caa,gold\")")
    ap.add_argument("--data_dir", default="data", help="Folder containing the four datasets")
    args = ap.parse_args()

    # Make indices (uses args.data_dir)
    os.environ["LUNAR_DATA_DIR"] = args.data_dir

    brief_path = Path(args.brief)
    brief = brief_path.read_text(encoding="utf-8")
    colors = parse_colors_arg(args.colors)

    build_indices()
    res = generate(brief, colors)
    jpath, mpath = save_output(res, brief_path.stem)
    print(f"Saved:\n  JSON: {jpath}\n  MD:   {mpath}")

if __name__ == "__main__":
    main()
""")

print("✅ Wrote files to", BASE)
print("Now run Cell 2 to build indices and Cell 3 for a demo.")


✅ Wrote files to /Users/ranykhirbawi/Desktop/LunarAIccord/data
Now run Cell 2 to build indices and Cell 3 for a demo.
